# Part 1: Distributed data parallel MNIST training with PyTorch and SageMaker distributed

## Background
[Amazon SageMaker's distributed library](https://docs.aws.amazon.com/sagemaker/latest/dg/distributed-training.html) can be used to train deep learning models faster and cheaper. The [data parallel](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel.html) feature in this library is a distributed data parallel training framework for PyTorch, TensorFlow, and MXNet. This notebook demonstrates how to use the SageMaker distributed data library to train a PyTorch model using the MNIST dataset.

This notebook example shows how to use `smdistributed.dataparallel` with PyTorch in SageMaker using MNIST dataset.

For more information:

1. [SageMaker distributed data parallel PyTorch API Specification](https://sagemaker.readthedocs.io/en/stable/api/training/smd_data_parallel_pytorch.html)
1. [Getting started with SageMaker distributed data parallel](https://sagemaker.readthedocs.io/en/stable/api/training/smd_data_parallel.html)
1. [PyTorch in SageMaker](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html)

### Dataset
This example uses the MNIST dataset. MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits).

**NOTE:** This example requires SageMaker Python SDK v2.**.

In [ ]:
!pip install sagemaker --upgrade

### SageMaker role

The following code cell defines `role` which is the IAM role ARN used to create and run SageMaker training and hosting jobs. This is the same IAM role used to create this SageMaker Notebook instance. 

`role` must have permission to create a SageMaker training job and launch an endpoint to host a model. For granular policies you can use to grant these permissions, see [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html).

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
role_name = role.split(["/"][-1])
print(f"The Amazon Resource Name (ARN) of the role used for this demo is: {role}")
print(f"The name of the role used for this demo is: {role_name[-1]}")

The Amazon Resource Name (ARN) of the role used for this demo is: arn:aws:iam::570106654206:role/ziyi-dev
The name of the role used for this demo is: ziyi-dev


To verify that the role above has required permissions:

1. Go to the IAM console: https://console.aws.amazon.com/iam/home.
2. Select **Roles**.
3. Enter the role name in the search box to search for that role. 
4. Select the role.
5. Use the **Permissions** tab to verify this role has required permissions attached.

## Model training with SageMaker distributed data parallel

### Training script

The MNIST dataset is downloaded using the `torchvision.datasets` PyTorch module; you can see how this is implemented in the `train_pytorch_smdataparallel_mnist.py` training script that is printed out in the next cell.

The training script provides the code you need for distributed data parallel (DDP) training using SageMaker's distributed data parallel library (`smdistributed.dataparallel`). The training script is very similar to a PyTorch training script you might run outside SageMaker, but modified to run with the `smdistributed.dataparallel` library. This library's PyTorch client provides an alternative to PyTorch's native DDP.

For details about how to use `smdistributed.dataparallel`'s DDP in your native PyTorch script, see the [Modify a PyTorch Training Script Using SMD Data Parallel](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel-modify-sdp.html#data-parallel-modify-sdp-pt).

In [ ]:
!pygmentize code/train_pytorch_smdataparallel_mnist.py

### Estimator function options

In the following code block, you can update the estimator function to use a different instance type, instance count, and distribution strategy. You're also passing in the training script you reviewed in the previous cell to this estimator.

**Instance types**

`smdistributed.dataparallel` supports model training on SageMaker with the following instance types only.  For best performance, it is recommended you use an instance type that supports Amazon Elastic Fabric Adapter (ml.p3dn.24xlarge and ml.p4d.24xlarge).

1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of `smdistributed.dataparallel`, you should use at least 2 instances, but you can also use 1 for testing this example.

**Distribution strategy**

Note that to use DDP mode, you update the `distribution` strategy, and set it to use `smdistributed dataparallel`.

In [103]:
from sagemaker.pytorch import PyTorch
from sagemaker.instance_group import InstanceGroup


group1 = InstanceGroup("spine1", "ml.c5n.2xlarge", 4)
group2 = InstanceGroup("spine2", "ml.c5n.2xlarge", 4)  


estimator = PyTorch(
    base_job_name="pytorch-smdataparallel-mnist",
    source_dir="code",
    entry_point="entry_point.py",
    role=role,
    framework_version="1.11.0",
    py_version="py38",
    # For training with multinode distributed training, set this count. Example: 2
    #instance_count=4,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    #instance_type="ml.c5n.2xlarge",
    instance_groups = [group1, group2],
    sagemaker_session=sagemaker_session,
    # Training using SMDataParallel Distributed Training Framework
    debugger_hook_config=False,
    #keep_alive_period_in_seconds=1800,
    hyperparameters = {'pp-degree': 4, 'dp-degree': 2,
                       'optimize-for-pp': 'False', 'dp-major': 'True', 'entry-point': 'hello.py',
                       'flag1' : 'flag1_val', 'flag2' : 'flag2_val', 'flag3': None}
)

In [ ]:
estimator.fit()

## Next steps

Now that you have a trained model, you can deploy an endpoint to host the model. After you deploy the endpoint, you can then test it with inference requests. The following cell will store the model_data variable to be used with the inference notebook.

In [ ]:
model_data = estimator.model_data
print("Storing {} as model_data".format(model_data))
%store model_data

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.instance_group import InstanceGroup

s3_train_bucket = "s3://viskaria/enwik8"

train = sagemaker.inputs.TrainingInput(
            s3_train_bucket, distribution="FullyReplicated", s3_data_type="S3Prefix"
        )
data_channels = {"train": train}

estimator = PyTorch(
    base_job_name="topology-inference-mpi",
    source_dir="code",
    entry_point="topology_aware_launcher.py",
    role=role,
    image_uri="570106654206.dkr.ecr.us-east-1.amazonaws.com/megatron-deepspeed:pt1.13.1-deeperspeed-0.8.3-v1",
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=8,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type="ml.p4d.24xlarge",
    sagemaker_session=sagemaker_session,
    # Training using SMDataParallel Distributed Training Framework
    debugger_hook_config=False,
    keep_alive_period_in_seconds=1800,
    hyperparameters = {'pp-degree': 4, 'dp-degree': 2,
                       'optimize-for-pp': 'False', 'dp-major': 'True', 
                       'entry-point': 'gpt-neox/train_torchrun.py', 
                       'conf': 'gpt-neox/sai_vishwa_15B.yml', 
                       }
)
estimator.fit(inputs=data_channels)

## Test Megatron+DS with gpt-neox 

In [11]:
# Configure FSx Input for your SageMaker Training job
from sagemaker.inputs import FileSystemInput

# FSx file system ID with your training dataset. Example: 'fs-0bYYYYYY'
file_system_id = "fs-058ddb4f9d002316a"
# FSx path with your training data # Example: '/fsx_mount_name/imagenet'
file_system_directory_path = "/evoynbev/enwik8"
file_system_access_mode = "rw"
file_system_type = "FSxLustre"
train_fs = FileSystemInput(
    file_system_id=file_system_id,
    file_system_type=file_system_type,
    directory_path=file_system_directory_path,
    file_system_access_mode=file_system_access_mode,
)
# Specify the training data channel using the FSx filesystem. This will be provided to the SageMaker training job later
data_channels = {"train": train_fs}

# The following variables will be used later in the notebook
subnets = ["subnet-8caa97c1"]  # Should be the subnet used for FSx. Example: subnet-0f9XXXX
security_group_ids = ["sg-fa5abbfb"]  # Should be the security group used for FSx. sg-03ZZZZZZ


In [16]:
s3_train_bucket = "s3://viskaria/enwik8"

train = sagemaker.inputs.TrainingInput(
            s3_train_bucket, distribution="FullyReplicated", s3_data_type="S3Prefix"
        )
data_channels = {"train": train}

In [ ]:
from sagemaker.pytorch import PyTorch
estimator = PyTorch(
    base_job_name="topology-inference-mpi",
    source_dir="code",
    entry_point="topology_aware_launcher.py",
    role=role,
    image_uri="570106654206.dkr.ecr.us-east-1.amazonaws.com/megatron-deepspeed:pt1.13.1-deeperspeed-0.8.3-v1",
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=16,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type="ml.p4d.24xlarge",
    sagemaker_session=sagemaker_session,
    # Training using SMDataParallel Distributed Training Framework
    debugger_hook_config=False,
    keep_alive_period_in_seconds=1800,
    hyperparameters = {'pp-degree': 2, 'dp-degree': 8,
                       'optimize-for-pp': 'True', 'dp-major': 'True', 
                       'entry-point': 'hello.py', 
                       'bad-placement': False,
                       'conf': 'gpt-neox/sai_vishwa_15B.yml', 
                       }
)
estimator.fit(inputs=data_channels)

INFO:sagemaker:Creating training-job with name: topology-inference-mpi-2023-05-01-04-50-21-390


Using provided s3_resource
2023-05-01 04:50:22 Starting - Starting the training job...
2023-05-01 04:50:44 Downloading - Downloading input data
2023-05-01 04:50:44 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-01 04:50:56,806 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-01 04:50:56,867 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-01 04:50:56,876 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-01 04:50:56,878 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-01 04:50:57,146 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-01 04:50:57,217 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neuron